<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2023/Team05/blob/main/spark_jobs_mark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


In [3]:
#install spark. we are using the one that uses hadoop as the underlying scheduler.
!wget -q https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf  spark-3.2.4-bin-hadoop3.2.tgz
!ls -l

#Provides findspark.init() to make pyspark importable as a regular library.
os.environ["SPARK_HOME"] = "spark-3.2.4-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 pyspark-shell'

total 588276
drwx------  6 root root      4096 Apr 29 17:43 drive
drwxr-xr-x  2 root root      4096 Apr 29 17:51 output.csv
drwxr-xr-x  1 root root      4096 Apr 27 13:35 sample_data
drwxr-xr-x 13 1000 1000      4096 Apr  9 21:17 spark-3.2.4-bin-hadoop3.2
-rw-r--r--  1 root root 301183180 Apr  9 21:35 spark-3.2.4-bin-hadoop3.2.tgz
-rw-r--r--  1 root root 301183180 Apr  9 21:35 spark-3.2.4-bin-hadoop3.2.tgz.1


In [4]:
!pip install -q findspark pyspark
import findspark
findspark.init()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!cat /content/drive/MyDrive/Big-Data-Final/nfd_incidents_xd_seg.parquet

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

## Average response time + variance by SegID

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("read_parquet_file").getOrCreate()

df = spark.read.parquet("/content/drive/MyDrive/Big-Data-Final/nfd_incidents_xd_seg.parquet")


In [7]:
print(df.show(5))

+--------------------+-----------+------------+-------------+--------------------+--------------------+-----------------+-----------+--------------+--------------------+-----------+------------------+-------------+
|         ID_Original|   latitude|   longitude|emdCardNumber|            time_utc|          time_local|response_time_sec|day_of_week|weekend_or_not|            geometry|Incident_ID|       Dist_to_Seg|      XDSegID|
+--------------------+-----------+------------+-------------+--------------------+--------------------+-----------------+-----------+--------------+--------------------+-----------+------------------+-------------+
|ObjectId(59d3a819...|36.03722849|-86.78324314|         29B5|2017-01-01 07:59:...|2017-01-01 01:59:...|            268.0|          6|             1|POINT (-86.783243...|         10|13.550370382347305|1.524393684E9|
|ObjectId(59d3a819...|36.03741402|-86.78657189|        29D2P| 2017-01-01 08:30:55| 2017-01-01 02:30:55|            512.0|          6|       

In [ ]:
#HERE WE HAVE TO CREATE THE CENSUS TRACT PER ROW

In [14]:
from pyspark.sql.functions import avg

# Read the input file as a Spark dataframe
# df = spark.read.format("csv").option("header", "true").load("input.csv")

# Group the dataframe by XDSegID and compute the average response time
result_df = df.groupBy("XDSegID").agg(avg("response_time_sec").alias("AvgResponseTime"))

# Write the result to a CSV file
result_df.write.format("csv").option("header", "true").mode("overwrite").save("output.csv")

In [15]:
result_df.show()

+-------------+------------------+
|      XDSegID|   AvgResponseTime|
+-------------+------------------+
|1.524470198E9|             349.2|
| 1.52446673E9|             404.0|
| 4.41552617E8| 297.6666666666667|
|  4.5042845E8|             582.5|
|1.524609417E9|             407.6|
|1.524475884E9|             337.0|
| 4.29334846E8|            490.25|
|1.524322123E9|             154.5|
| 4.49620825E8|             553.0|
|1.524554221E9|             410.0|
| 4.49621869E8|             798.0|
| 4.49629818E8|             148.5|
| 1.56811863E8|             405.0|
|1.524576615E9|             496.0|
| 4.29334039E8|             449.6|
| 1.56551371E8|             256.0|
| 1.56121138E8|             310.8|
|1.524303919E9|           242.625|
|1.524378121E9|476.14285714285717|
|1.524474299E9|             449.5|
+-------------+------------------+
only showing top 20 rows



In [17]:
from pyspark.sql.functions import col, variance

# group by XDSegID and calculate variance of ResponseTime
df_agg = df.groupBy("XDSegID").agg(variance(col("response_time_sec")).alias("Variance"))

# show the results
df_agg.show()

+-------------+------------------+
|      XDSegID|          Variance|
+-------------+------------------+
|1.524470198E9|           51834.2|
| 1.52446673E9| 68509.06666666667|
| 4.41552617E8| 4837.333333333333|
|  4.5042845E8|          180600.5|
|1.524609417E9|           23415.3|
|1.524475884E9|              null|
| 4.29334846E8|         124896.25|
|1.524322123E9|             364.5|
| 4.49620825E8|           18050.0|
|1.524554221E9|              null|
| 4.49621869E8|              null|
| 4.49629818E8|           42340.5|
| 1.56811863E8|              null|
|1.524576615E9|              null|
| 4.29334039E8|11660.300000000001|
| 1.56551371E8|            2584.0|
| 1.56121138E8|           20254.7|
|1.524303919E9|14160.916666666664|
|1.524378121E9|18207.476190476194|
|1.524474299E9|           19012.5|
+-------------+------------------+
only showing top 20 rows



## Weather correlation on the the incidents and response time distribution

In [19]:
from pyspark.sql.functions import min, max, date_format

min_local_time = df.select(min(date_format('time_local', 'yyyy-MM-dd HH:mm:ss'))).collect()[0][0]
max_local_time = df.select(max(date_format('time_local', 'yyyy-MM-dd HH:mm:ss'))).collect()[0][0]

print("Minimum local time: ", min_local_time)
print("Maximum local time: ", max_local_time)


Minimum local time:  2017-01-01 01:59:29
Maximum local time:  2021-03-02 07:45:57


In [20]:
weather_df = spark.read.parquet("/content/drive/MyDrive/Big-Data-Final/weather_tn.parquet/year=2017/month=1/part-00039-0c8142a9-ffb0-4186-8d5f-060d4d2afe7d.c000.snappy.parquet")

In [22]:
weather_df.show(5)

+------------+-------------+-----------+-------------------+----+--------+-------------------+---+----------+--------+----------+---------+-----+-------+-----+----+---+-------+--------+---+---+----+---+-------------+----+------+------+-----------+----+-----+---------------+-----------------------+------------------------+-----------+----------+
|  station_id|start_date_st|end_date_st|    timestamp_local|  rh|wind_spd|      timestamp_utc|pod|       slp|app_temp|elev_angle|solar_rad| pres|h_angle|dewpt|snow| uv|azimuth|wind_dir|ghi|dhi| vis|dni|     datetime|temp|precip|clouds|         ts|icon| code|    description|gps_coordinate_latitude|gps_coordinate_longitude| spatial_id|      days|
+------------+-------------+-----------+-------------------+----+--------+-------------------+---+----------+--------+----------+---------+-----+-------+-----+----+---+-------+--------+---+---+----+---+-------------+----+------+------+-----------+----+-----+---------------+-----------------------+--------

In [44]:
df.show(5)

+--------------------+-----------+------------+-------------+--------------------+----------+-----------------+-----------+--------------+--------------------+-----------+------------------+-------------+-------------------+-------------------+----------+
|         ID_Original|   latitude|   longitude|emdCardNumber|            time_utc|time_local|response_time_sec|day_of_week|weekend_or_not|            geometry|Incident_ID|       Dist_to_Seg|      XDSegID|     time_local_str|    timestamp_local|      date|
+--------------------+-----------+------------+-------------+--------------------+----------+-----------------+-----------+--------------+--------------------+-----------+------------------+-------------+-------------------+-------------------+----------+
|ObjectId(59d3a819...|36.03722849|-86.78324314|         29B5|2017-01-01 07:59:...|1483235969|            268.0|          6|             1|POINT (-86.783243...|         10|13.550370382347305|1.524393684E9|2017-01-01 01:59:29|2017-01-

In [74]:
import requests
import json
from pyspark.sql.functions import to_timestamp, unix_timestamp, from_unixtime

# define your OpenWeatherMap API key
api_key = "7efb75d5c7c543ecbecd97c26e4fa62e"

# define the endpoint URL for the API call
endpoint_url = "https://api.openweathermap.org/data/2.5/weather"

df = df.withColumn('time_local_str', from_unixtime(df['time_local'], 'yyyy-MM-dd HH:mm:ss'))
df = df.withColumn("timestamp_local", from_unixtime(df.time_local)) \
       .withColumn("date", date_format("timestamp_local", "yyyy-MM-dd"))

# define the columns you want to join on
join_columns = ["time_local", "latitude", "longitude"]



df.show(5)


+--------------------+-----------+------------+-------------+--------------------+----------+-----------------+-----------+--------------+--------------------+-----------+------------------+-------------+-------------------+-------------------+----------+
|         ID_Original|   latitude|   longitude|emdCardNumber|            time_utc|time_local|response_time_sec|day_of_week|weekend_or_not|            geometry|Incident_ID|       Dist_to_Seg|      XDSegID|     time_local_str|    timestamp_local|      date|
+--------------------+-----------+------------+-------------+--------------------+----------+-----------------+-----------+--------------+--------------------+-----------+------------------+-------------+-------------------+-------------------+----------+
|ObjectId(59d3a819...|36.03722849|-86.78324314|         29B5|2017-01-01 07:59:...|1483235969|            268.0|          6|             1|POINT (-86.783243...|         10|13.550370382347305|1.524393684E9|2017-01-01 01:59:29|2017-01-

In [75]:
import requests
import json
from pyspark.sql.functions import from_unixtime

# function to query the OpenWeatherMap API for weather conditions at the specified time and location
def get_weather_data(row):
    params = {
        "lat": row["latitude"],
        "lon": row["longitude"],
        "dt": row["time_local"],
        "appid": api_key,
    }
    response = requests.get(endpoint_url, params=params)
    weather_data = json.loads(response.text)
    print(response.text)
    if response is None:
      return None
    else:
      return (row["time_local"], row["latitude"], row["longitude"], weather_data.get("weather")[0].get("main"))

# create a new DataFrame with weather data joined to the original DataFrame
join_columns = ["time_local", "latitude", "longitude"]


import time

batch_size = 100
batch_count = df.count() // batch_size + 1

for i in range(batch_count):
    start_index = i * batch_size
    end_index = (i + 1) * batch_size

    # create batch dataframe
    batch_df = df.limit(batch_size).toPandas()[start_index:end_index]
    
    # process batch and join weather data
    batch_weather_df = batch_df.apply(get_weather_data, axis=1)
    batch_weather_df.columns = ["time_local", "latitude", "longitude", "weather"]
    joined_df = batch_df.join(batch_weather_df.set_index(["time_local", "latitude", "longitude"]), on=["time_local", "latitude", "longitude"])
    
    # do further processing on the joined_df
    print("next one")
    time.sleep(60) # wait for a minute between each batch to avoid exceeding the rate limit


{"cod":429, "message": "Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription https://openweathermap.org/price"}


/content/spark-3.2.4-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


TypeError: ignored

Py4JJavaError: ignored

Trying Again with Local Stuff

In [1]:
import os
import pandas as pd

# specify the root directory where parquet files are located
root_dir = '/content/drive/MyDrive/Big-Data-Final/weather_tn.parquet'

# create an empty list to store the dataframes
dfs = []

# recursively search through the directory tree and find all parquet files
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.parquet'):
            # read in the parquet file and append it to the list of dataframes
            filepath = os.path.join(subdir, file)
            df = pd.read_parquet(filepath)

            # group the DataFrame by the timestamp column
            grouped = df.groupby('timestamp_local')

            # filter the groups to only keep those with 5 or more rows
            filtered = grouped.filter(lambda x: len(x) >= 5)

            # sample 5 rows from each group
            sampled = filtered.groupby('timestamp_local').apply(lambda x: x.sample(n=5))

            # reset the index of the resulting DataFrame
            result = sampled.reset_index(drop=True)

            dfs.append(result)

# concatenate all of the dataframes into a single dataframe
merged_df = pd.concat(dfs, ignore_index=True)


In [2]:
merged_df

,station_id,start_date_st,end_date_st,timestamp_local,rh,wind_spd,timestamp_utc,pod,slp,app_temp,...,precip,clouds,ts,icon,code,description,gps_coordinate_latitude,gps_coordinate_longitude,spatial_id,days
0,722054-99999,None,None,2021-12-01 00:00:00,78.0,2.6,2021-12-01 06:00:00,n,1018.2000,10.0,...,0.0,0.0,1.638338e+09,c01n,800.0,Clear Sky,35.135,-90.234,West Memphis,2021-12-01
1,723270-13897,None,None,2021-12-01 00:00:00,76.0,2.4,2021-12-01 06:00:00,n,1020.4000,4.3,...,0.0,25.0,1.638338e+09,c02n,802.0,Scattered clouds,36.119,-86.689,Berry Hill,2021-12-01
2,726381-99999,None,None,2021-12-01 00:00:00,93.0,2.4,2021-12-01 06:00:00,n,1020.3000,0.6,...,0.0,0.0,1.638338e+09,c01n,800.0,Clear Sky,34.933,-86.083,Hytop,2021-12-01
3,KUCY,None,None,2021-12-01 00:00:00,100.0,2.4,2021-12-01 06:00:00,n,1019.0000,1.8,...,0.0,0.0,1.638338e+09,c01n,800.0,Clear Sky,36.380,-88.985,Rives,2021-12-01
4,723280-03894,None,None,2021-12-01 00:00:00,85.0,0.8,2021-12-01 06:00:00,n,1019.8000,4.5,...,0.0,0.0,1.638338e+09,c01n,800.0,Clear Sky,36.624,-87.419,Oak Grove,2021-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219090,722177-63811,None,None,2020-07-31 23:00:00,100.0,0.8,2020-08-01 03:00:00,n,1017.5304,22.1,...,0.0,87.0,1.596251e+09,c04n,804.0,Overcast clouds,35.195,-83.865,Andrews,2020-07-31
219091,722322-99999,None,None,2020-07-31 23:00:00,100.0,2.4,2020-08-01 04:00:00,n,1007.6550,23.4,...,0.5,99.0,1.596254e+09,c04n,804.0,Overcast clouds,37.032,-85.954,Glasgow,2020-07-31
219092,720481-99999,None,None,2020-07-31 23:00:00,89.0,0.8,2020-08-01 04:00:00,n,1008.0000,24.4,...,0.5,38.0,1.596254e+09,c02n,802.0,Scattered clouds,36.601,-89.992,Malden,2020-07-31
219093,746710-13806,None,None,2020-07-31 23:00:00,100.0,3.1,2020-08-01 04:00:00,n,1009.8000,24.0,...,2.0,96.0,1.596254e+09,r01n,500.0,Light rain,36.667,-87.483,Fort Campbell North,2020-07-31


In [14]:
merged_df['timestamp_local'].min()
merged_df['timestamp_local'].max()

Timestamp('2021-12-31 23:00:00')

In [ ]:
import pandas as pd

# assume 'df' is the DataFrame with the timestamp column called 'timestamp_col'

# group the DataFrame by the timestamp column
grouped = merged_df.groupby('timestamp_local')

# filter the groups to only keep those with 5 or more rows
filtered = grouped.filter(lambda x: len(x) >= 5)

# sample 5 rows from each group
sampled = filtered.groupby('timestamp_local').apply(lambda x: x.sample(n=5))

# reset the index of the resulting DataFrame
result = sampled.reset_index(drop=True)


In [6]:
pandas_df = df.toPandas()
pandas_df


/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,ID_Original,latitude,longitude,emdCardNumber,time_utc,time_local,response_time_sec,day_of_week,weekend_or_not,geometry,Incident_ID,Dist_to_Seg,XDSegID
0,ObjectId(59d3a81908f47311c891f8e2),36.037228,-86.783243,29B5,2017-01-01 07:59:29.507,2017-01-01 01:59:29.507,268.0,6,1,POINT (-86.78324314 36.03722849),10,13.550370,1.524394e+09
1,ObjectId(59d3a81908f47311c891f8e6),36.037414,-86.786572,29D2P,2017-01-01 08:30:55.000,2017-01-01 02:30:55.000,512.0,6,1,POINT (-86.78657189 36.03741402),11,3.663952,1.524356e+09
2,ObjectId(59d3a81a08f47311c891f8ef),36.217663,-86.808699,29D1H,2017-01-01 09:35:10.000,2017-01-01 03:35:10.000,271.0,6,1,POINT (-86.80869908 36.21766267),12,NaN,NaN
3,ObjectId(59d3a81a08f47311c891f8f9),36.057790,-86.739728,29D8V,2017-01-01 10:29:31.867,2017-01-01 04:29:31.867,366.0,6,1,POINT (-86.73972836 36.05779026),13,NaN,NaN
4,ObjectId(59d3a81a08f47311c891f904),36.039221,-86.602785,29A2V,2017-01-01 11:09:13.000,2017-01-01 05:09:13.000,447.0,6,1,POINT (-86.60278517 36.03922079),14,3.171782,4.496208e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29760,ObjectId(603dab3d95fad0761f05c089),36.264172,-86.727176,29A2V,2021-03-02 03:00:41.943,2021-03-01 21:00:41.943,401.0,0,0,POINT (-86.72717565 36.26417165),29770,5.601240,4.496178e+08
29761,ObjectId(603dab3f95fad0761f05c08a),36.045373,-86.658851,29B1,2021-03-02 03:01:57.740,2021-03-01 21:01:57.740,985.0,0,0,POINT (-86.65885054 36.04537322),29771,1.961266,1.524314e+09
29762,ObjectId(603daec495fad0761f05c08d),36.116022,-86.782203,29B5,2021-03-02 03:16:37.113,2021-03-01 21:16:37.113,NaN,0,0,POINT (-86.78220264 36.11602207),29772,3.013125,1.524340e+09
29763,ObjectId(603e2d5795fad0761f05c0f2),36.159411,-86.822013,29B1,2021-03-02 12:15:58.047,2021-03-02 06:15:58.047,791.0,1,0,POINT (-86.82201339 36.15941116),29773,3.195641,1.524408e+09


In [10]:
# convert the timestamp column to a datetime object
pandas_df['time_local'] = pd.to_datetime(pandas_df['time_local'])
merged_df['timestamp_local'] = pd.to_datetime(merged_df['timestamp_local'])

# add a new column to both dataframes with the hour part of the timestamp
pandas_df['hour'] = pandas_df['time_local'].dt.hour
merged_df['hour'] = merged_df['timestamp_local'].dt.hour

pandas_df['date'] = pandas_df['time_local'].dt.date
merged_df['date'] = merged_df['timestamp_local'].dt.date

In [12]:
pandas_df

,ID_Original,latitude,longitude,emdCardNumber,time_utc,time_local,response_time_sec,day_of_week,weekend_or_not,geometry,Incident_ID,Dist_to_Seg,XDSegID,hour,date
0,ObjectId(59d3a81908f47311c891f8e2),36.037228,-86.783243,29B5,2017-01-01 07:59:29.507,2017-01-01 01:59:29.507,268.0,6,1,POINT (-86.78324314 36.03722849),10,13.550370,1.524394e+09,1,2017-01-01
1,ObjectId(59d3a81908f47311c891f8e6),36.037414,-86.786572,29D2P,2017-01-01 08:30:55.000,2017-01-01 02:30:55.000,512.0,6,1,POINT (-86.78657189 36.03741402),11,3.663952,1.524356e+09,2,2017-01-01
2,ObjectId(59d3a81a08f47311c891f8ef),36.217663,-86.808699,29D1H,2017-01-01 09:35:10.000,2017-01-01 03:35:10.000,271.0,6,1,POINT (-86.80869908 36.21766267),12,NaN,NaN,3,2017-01-01
3,ObjectId(59d3a81a08f47311c891f8f9),36.057790,-86.739728,29D8V,2017-01-01 10:29:31.867,2017-01-01 04:29:31.867,366.0,6,1,POINT (-86.73972836 36.05779026),13,NaN,NaN,4,2017-01-01
4,ObjectId(59d3a81a08f47311c891f904),36.039221,-86.602785,29A2V,2017-01-01 11:09:13.000,2017-01-01 05:09:13.000,447.0,6,1,POINT (-86.60278517 36.03922079),14,3.171782,4.496208e+08,5,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29760,ObjectId(603dab3d95fad0761f05c089),36.264172,-86.727176,29A2V,2021-03-02 03:00:41.943,2021-03-01 21:00:41.943,401.0,0,0,POINT (-86.72717565 36.26417165),29770,5.601240,4.496178e+08,21,2021-03-01
29761,ObjectId(603dab3f95fad0761f05c08a),36.045373,-86.658851,29B1,2021-03-02 03:01:57.740,2021-03-01 21:01:57.740,985.0,0,0,POINT (-86.65885054 36.04537322),29771,1.961266,1.524314e+09,21,2021-03-01
29762,ObjectId(603daec495fad0761f05c08d),36.116022,-86.782203,29B5,2021-03-02 03:16:37.113,2021-03-01 21:16:37.113,NaN,0,0,POINT (-86.78220264 36.11602207),29772,3.013125,1.524340e+09,21,2021-03-01
29763,ObjectId(603e2d5795fad0761f05c0f2),36.159411,-86.822013,29B1,2021-03-02 12:15:58.047,2021-03-02 06:15:58.047,791.0,1,0,POINT (-86.82201339 36.15941116),29773,3.195641,1.524408e+09,6,2021-03-02


In [13]:

# merge the dataframes on the hour and any other columns you need
merged = pd.merge(pandas_df, merged_df, on=['hour', 'date'], how='inner')

# drop the 'hour' column from the merged dataframe
merged = merged.drop(columns=['hour', 'date'])


#final_df = pd.merge(pandas_df, merged_df, left_on='time_local', right_on='timestamp_local')


In [12]:
final_df[['time_local', 'timestamp_local']]

,time_local,timestamp_local
0,2017-03-01 01:00:00,2017-03-01 01:00:00
1,2017-03-01 01:00:00,2017-03-01 01:00:00
2,2017-03-01 01:00:00,2017-03-01 01:00:00
3,2017-03-01 01:00:00,2017-03-01 01:00:00
4,2017-03-01 01:00:00,2017-03-01 01:00:00


In [18]:
merged

,ID_Original,latitude,longitude,emdCardNumber,time_utc,time_local,response_time_sec,day_of_week,weekend_or_not,geometry,...,precip,clouds,ts,icon,code,description,gps_coordinate_latitude,gps_coordinate_longitude,spatial_id,days
0,ObjectId(59d3a81908f47311c891f8e2),36.037228,-86.783243,29B5,2017-01-01 07:59:29.507,2017-01-01 01:59:29.507,268.0,6,1,POINT (-86.78324314 36.03722849),...,0.0,100.0,1.483254e+09,c04n,804.0,Overcast clouds,35.380,-86.246,Tullahoma,2017-01-01
1,ObjectId(59d3a81908f47311c891f8e2),36.037228,-86.783243,29B5,2017-01-01 07:59:29.507,2017-01-01 01:59:29.507,268.0,6,1,POINT (-86.78324314 36.03722849),...,0.5,100.0,1.483250e+09,c04n,804.0,Overcast clouds,35.818,-83.986,Alcoa,2017-01-01
2,ObjectId(59d3a81908f47311c891f8e2),36.037228,-86.783243,29B5,2017-01-01 07:59:29.507,2017-01-01 01:59:29.507,268.0,6,1,POINT (-86.78324314 36.03722849),...,0.5,100.0,1.483254e+09,c04n,804.0,Overcast clouds,36.119,-86.689,Berry Hill,2017-01-01
3,ObjectId(59d3a81908f47311c891f8e2),36.037228,-86.783243,29B5,2017-01-01 07:59:29.507,2017-01-01 01:59:29.507,268.0,6,1,POINT (-86.78324314 36.03722849),...,0.0,50.0,1.483250e+09,c03n,803.0,Broken clouds,36.200,-81.650,Boone,2017-01-01
4,ObjectId(59d3a81908f47311c891f8e2),36.037228,-86.783243,29B5,2017-01-01 07:59:29.507,2017-01-01 01:59:29.507,268.0,6,1,POINT (-86.78324314 36.03722849),...,0.0,100.0,1.483250e+09,c04n,804.0,Overcast clouds,36.430,-81.420,Jefferson,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148820,ObjectId(603e427795fad0761f05c102),36.051939,-86.670056,29B1,2021-03-02 13:45:57.253,2021-03-02 07:45:57.253,542.0,1,0,POINT (-86.6700563 36.05193902),...,0.0,0.0,1.614690e+09,c01d,800.0,Clear Sky,36.665,-88.373,Murray,2021-03-02
148821,ObjectId(603e427795fad0761f05c102),36.051939,-86.670056,29B1,2021-03-02 13:45:57.253,2021-03-02 07:45:57.253,542.0,1,0,POINT (-86.6700563 36.05193902),...,0.0,0.0,1.614690e+09,c01d,800.0,Clear Sky,35.967,-89.950,Gosnell,2021-03-02
148822,ObjectId(603e427795fad0761f05c102),36.051939,-86.670056,29B1,2021-03-02 13:45:57.253,2021-03-02 07:45:57.253,542.0,1,0,POINT (-86.6700563 36.05193902),...,0.0,0.0,1.614690e+09,c01d,800.0,Clear Sky,35.593,-88.917,Jackson,2021-03-02
148823,ObjectId(603e427795fad0761f05c102),36.051939,-86.670056,29B1,2021-03-02 13:45:57.253,2021-03-02 07:45:57.253,542.0,1,0,POINT (-86.6700563 36.05193902),...,0.0,87.0,1.614690e+09,c04d,804.0,Overcast clouds,36.119,-86.689,Berry Hill,2021-03-02


In [ ]:
#PICK the closest weather station as that incident's weather OR just pick one on the join?
#plot by categorical weather condtions and their response time/incident frequency
# machine learning on the numerical attributes like precipataion etc for biggest contributors
